<a href="https://colab.research.google.com/github/Sandell0/Colab-experiments/blob/master/Learn_sha_and_back_using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!export WANDB_PROJECT="sha_hacker"
WANDB_PROJECT="sha_hacker"

In [ ]:
%pip install transformers
%pip install datasets
%pip install wandb

In [ ]:
import torch
import tqdm
from datasets import load_dataset, Dataset
from transformers import (
    EncoderDecoderModel,
    AutoTokenizer,
    BatchEncoding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    T5ForConditionalGeneration
)
import random
import string
import hashlib

In [ ]:
import transformers
transformers.utils.logging.set_verbosity_warning()

In [ ]:
string_size=64
str1 = "1" * string_size
result = hashlib.sha1(str1.encode())
print(bin(int(result.hexdigest(),16))[2:].zfill(256))

In [ ]:
my_dict = {"text":[],"code":[]}
for i in range(3):
  custom_string = ''.join(random.choice(string.printable) for _ in range(string_size))
  my_dict["text"].append(custom_string)
  my_dict["code"].append(bin(int(hashlib.sha256(custom_string.encode()).hexdigest(),16))[2:].zfill(256))


In [ ]:
data = Dataset.from_dict(my_dict)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/byt5-base")

def tokenize_function(examples):
    tokenized_text = tokenizer(
        examples["text"],
        padding="longest",
        max_length=256,
    )
    tokenized_code = tokenizer(
        examples["code"],
        padding="longest",
        max_length=256,
    )

    examples["input_ids"] = tokenized_code["input_ids"].copy()
    examples["labels"] = tokenized_text["input_ids"].copy()

    return examples

In [ ]:
torch.cuda.empty_cache()

class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
      prompts = [
          "0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011110100001101001010000101001100010011100111110011010111100001011101011100101001101011001000111100000010000110011011100010110110010011101011011100100100100",
      ]


      # encode prompt
      encoded_prompts = tokenizer(prompts, return_tensors="pt")

      encoded_prompts = encoded_prompts.to('cuda:0')

      # generate text
      outputs = model.generate(
          input_ids=encoded_prompts["input_ids"],
          max_length=256,
      )

      print(tokenizer.decode(outputs[0]).replace('\n',''))


train_test = data.train_test_split(test_size=0.001, shuffle=False)

for key in ["train", "test"]:
    train_test[key] = train_test[key].map(tokenize_function)
    print(len(train_test[key]), key)
model = T5ForConditionalGeneration.from_pretrained('google/byt5-base') # initialize Bert2Bert from pre-trained checkpoints


training_args = TrainingArguments(
    output_dir="./sha-hacker",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=3000,  # number of training epochs
    per_device_train_batch_size=2,  # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps=200,  # Number of update steps between two evaluations.
    save_steps=1000,  # after # steps model is saved
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    logging_strategy="steps",
    logging_steps=200,
    save_total_limit=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    learning_rate=1e-4,
    weight_decay=0.05,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    callbacks=[MyCallback],
)
print(train_test["train"][0])

trainer.train()
trainer.save_model()